In [1]:
!pip install lazypredict

     |████████████████████████████████| 1.7 MB 830 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 297.1 MB 8.5 kB/s eta 0:00:01
  Created wheel for lightgbm: filename=lightgbm-4.2.0-py3-none-manylinux_2_31_x86_64.whl size=2774572 sha256=12bacbbd48905d41d9b1829ba35d3673ab424b5a3cdbb46d27e493f089457264
  Stored in directory: /home/almaz/.cache/pip/wheels/94/dc/e1/c89606d16c7acd6e7ea1c0396ad6a933057855dcd8727d4309
Successfully built lightgbm


In [2]:
import lazypredict
from lazypredict.Supervised import LazyClassifier

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.model_selection import train_test_split
from DataPreparation import DataPreparation

In [3]:
df = pd.read_csv('./data/balanced_dataframe.csv', index_col=None)
df.head()

,MaxAbsEStateIndex,MinAbsEStateIndex,qed,SPS,MaxPartialCharge,MinPartialCharge,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_MWLOW,BCUT2D_CHGHI,...,fr_Nhpyrrole,fr_aldehyde,fr_amidine,fr_aryl_methyl,fr_benzene,fr_benzodiazepine,fr_diazo,fr_halogen,fr_hdrzine,label
0,10.34,-1.27,23.38,505.33,79.92,10.12,9.10,-0.05,3.46,1.38,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1
1,10.34,-1.27,23.38,505.33,79.92,10.12,9.10,-0.05,3.45,1.37,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1
2,10.33,-1.27,23.38,505.33,79.92,10.12,9.10,-0.05,3.45,1.36,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1
3,10.36,-1.24,25.22,440.50,16.56,9.90,5.83,-0.05,3.35,1.32,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1
4,6.09,0.23,12.29,362.61,79.92,10.10,9.10,0.55,3.02,1.98,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1


In [4]:
X = df.drop(columns=['label'])
y = df['label']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42,
                                                    stratify=y
                                                    )
y_test.value_counts()

0    915
1    915
Name: label, dtype: int64

In [6]:
data_preparation = DataPreparation(X_train, X_test, y_train, y_test)

# # get scaled clean data
# # use StandartScaler
X_train, X_test, y_train, y_test = data_preparation.clean_dataset()

Index(['MaxAbsEStateIndex', 'MinAbsEStateIndex', 'qed', 'SPS',
       'MaxPartialCharge', 'MinPartialCharge', 'BCUT2D_MWHI', 'BCUT2D_MWLOW',
       'BCUT2D_CHGHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW',
       'AvgIpc', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1',
       'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v',
       'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'PEOE_VSA1',
       'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA2', 'PEOE_VSA3',
       'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA8', 'SMR_VSA2', 'SMR_VSA3',
       'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA8', 'SMR_VSA9',
       'SlogP_VSA12', 'SlogP_VSA2', 'SlogP_VSA3', 'SlogP_VSA4', 'SlogP_VSA5',
       'SlogP_VSA6', 'SlogP_VSA8', 'TPSA', 'EState_VSA1', 'EState_VSA5',
       'VSA_EState1', 'VSA_EState6', 'VSA_EState7', 'NumAliphaticCarbocycles',
       'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumHeteroatoms',
       'NumRotatableBonds', 

In [7]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

 97%|████████████████████████████████████████▌ | 28/29 [00:21<00:00,  1.97it/s]

[LightGBM] [Info] Number of positive: 2135, number of negative: 2135
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004520 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11984
[LightGBM] [Info] Number of data points in the train set: 4270, number of used features: 83
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████████████████████████████████████| 29/29 [00:22<00:00,  1.28it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
XGBClassifier                      0.98               0.98     0.98      0.98   
RandomForestClassifier             0.98               0.98     0.98      0.98   
ExtraTreesClassifier               0.98               0.98     0.98      0.98   
LGBMClassifier                     0.97               0.97     0.97      0.97   
SVC                                0.97               0.97     0.97      0.97   
BaggingClassifier                  0.97               0.97     0.97      0.97   
LabelPropagation                   0.97               0.97     0.97      0.97   
LabelSpreading                     0.97               0.97     0.97      0.97   
DecisionTreeClassifier             0.95               0.95     0.95      0.95   
KNeighborsClassifier               0.95               0.95     0.95      0.95   
LogisticRegression          

## After feature selection N = 20

In [8]:
clmns_sel = ['PEOE_VSA13',
    'Chi3v',
    'FpDensityMorgan3',
    'Kappa1',
    'BertzCT',
    'EState_VSA3',
    'SMR_VSA4',
    'BCUT2D_LOGPLOW',
    'Chi2n',
    'qed',
    'EState_VSA11',
    'BCUT2D_MWHI',
    'MinPartialCharge',
    'MinAbsEStateIndex',
    'SMR_VSA2',
    'SlogP_VSA3',
    'Kappa3',
    'MaxPartialCharge',
    'SlogP_VSA1',
    'PEOE_VSA3' ]
R = X[clmns_sel]
R

,PEOE_VSA13,Chi3v,FpDensityMorgan3,Kappa1,BertzCT,EState_VSA3,SMR_VSA4,BCUT2D_LOGPLOW,Chi2n,qed,EState_VSA11,BCUT2D_MWHI,MinPartialCharge,MinAbsEStateIndex,SMR_VSA2,SlogP_VSA3,Kappa3,MaxPartialCharge,SlogP_VSA1,PEOE_VSA3
0,15.93,24.79,9.10,30.96,0.00,4.00,20.94,30683496.34,17.70,23.38,1.00,-0.05,10.12,-1.27,17.48,0.12,40.76,79.92,13.78,0.00
1,15.93,24.79,9.10,30.96,0.00,4.00,20.94,30010857.76,23.76,23.38,1.00,-0.05,10.12,-1.27,11.83,0.12,40.76,79.92,13.77,0.00
2,15.93,24.79,9.10,30.96,0.00,4.00,20.94,29891741.90,17.70,23.38,1.00,-0.05,10.12,-1.27,11.83,0.12,40.76,79.92,13.77,0.00
3,0.00,20.06,5.83,56.64,0.00,3.00,12.84,32433354.32,48.79,25.22,1.00,-0.05,9.90,-1.24,5.82,0.55,41.22,16.56,7.11,0.00
4,27.53,4.42,9.10,6.42,0.00,3.00,28.94,116765.32,45.76,12.29,1.00,0.55,10.10,0.23,5.95,0.90,51.15,79.92,6.27,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6095,11.34,0.00,7.22,27.32,5.56,3.00,15.78,168802.82,30.54,11.30,1.00,0.49,10.21,-0.41,11.48,0.58,50.67,32.13,2.26,12.84
6096,0.00,18.22,7.93,19.89,0.00,0.00,5.70,272147.62,13.85,29.38,0.00,-0.85,10.23,-3.54,5.91,-0.02,22.38,32.23,27.15,0.00
6097,11.60,19.43,6.32,45.57,5.91,2.00,57.68,119394275.58,38.82,22.54,0.00,-0.67,9.70,-0.20,29.19,1.14,40.37,35.50,5.24,25.18
6098,11.76,14.33,8.00,17.33,0.00,2.00,24.73,4473417.99,19.06,18.25,1.00,-0.13,10.30,-0.46,11.94,0.51,30.33,32.17,7.41,0.00


In [9]:
X_train, X_test, y_train, y_test = train_test_split(R, y, 
                                                    test_size=0.3, 
                                                    random_state=42,
                                                    stratify=y
                                                    )

In [10]:
data_preparation = DataPreparation(X_train, X_test, y_train, y_test)
X_train, X_test, y_train, y_test = data_preparation.clean_dataset()

Index(['PEOE_VSA13', 'Chi3v', 'FpDensityMorgan3', 'Kappa1', 'BertzCT',
       'EState_VSA3', 'SMR_VSA4', 'BCUT2D_LOGPLOW', 'Chi2n', 'qed',
       'EState_VSA11', 'BCUT2D_MWHI', 'MinPartialCharge', 'MinAbsEStateIndex',
       'SMR_VSA2', 'SlogP_VSA3', 'PEOE_VSA3'],
      dtype='object')


### Only 17 features left after cleaning

In [11]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

 97%|████████████████████████████████████████▌ | 28/29 [00:15<00:00,  3.09it/s]

[LightGBM] [Info] Number of positive: 2135, number of negative: 2135
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000466 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 4270, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████████████████████████████████████| 29/29 [00:15<00:00,  1.86it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.96               0.96     0.96      0.96   
XGBClassifier                      0.95               0.95     0.95      0.95   
RandomForestClassifier             0.95               0.95     0.95      0.95   
LGBMClassifier                     0.95               0.95     0.95      0.95   
BaggingClassifier                  0.94               0.94     0.94      0.94   
SVC                                0.93               0.93     0.93      0.93   
LabelSpreading                     0.92               0.92     0.92      0.92   
LabelPropagation                   0.92               0.92     0.92      0.92   
KNeighborsClassifier               0.91               0.91     0.91      0.91   
AdaBoostClassifier                 0.91               0.91     0.91      0.91   
DecisionTreeClassifier      